## Participants: 

#### Mohamed Afif Chifaoui (100452024)

#### Ricardo Vazquez Alvarez (100417117)

---

# Linear and Discrete Models HW1


## Problem statement:

A service company operates in three shifts (Morning, Afternoon, Night) and needs to determine the number of employees assigned to each shift. The objective is to minimize labor costs while ensuring sufficient staffing levels for service demand.

Firstly, we can determine that there are two types of sets for the model (the sets of shifts and the employee types). 

1. $I$: Set of Shifts (Morning, Afternoon, Nights)
2. $J$: Set of employee types (Regular, Part-time, Temporary, On-call, Training, Manager, Security).

The parameters to be used in the objective function and the constraints will be the following:

1. $D_{i}$: Service demand for each shift. ($s \in S$)
2. $E_{j}$: Available number of employees of each type. ($e \in E$)
3. $C_{j}$: Cost per hour for each employee type. ($c \in C$)



The variables are those employees with their type ($j$) to be assigned to a shift ($i$).
- $X_{ij}$: Number of employees of type $j$ assigned to shift $i$.

Leading to the Objective function:

$$\text{Minimize} \;\; Z = \sum_{i\in I}^{}\sum_{j\in J}^{} C_{j}\cdot X_{ij}$$



### Decision Variables:

The following are the individual decision variables that will be used:

- $x_{11}$: Number of  Regular employees assigned to the Morning shift.
- $x_{12}$: Number of Part-Time employees assigned to the Morning shift.
- $x_{13}$: Number of Temporary employees assigned to the Morning shift.
- $x_{14}$: Number of On-call employees assigned to the Morning shift.
- $x_{15}$: Number of Training employees assigned to the Morning shift.
- $x_{16}$: Number of Manager employees assigned to the Morning shift.
- $x_{17}$: Number of Security employees assigned to the Morning shift.
- $x_{21}$: Number of  Regular employees assigned to the Afternoon shift.
- $x_{22}$: Number of Part-Time employees assigned to the Afternoon shift.
- $x_{23}$: Number of Temporary employees assigned to the Afternoon shift.
- $x_{24}$: Number of On-call employees assigned to the Afternoon shift.
- $x_{25}$: Number of Training employees assigned to the Afternoon shift.
- $x_{26}$: Number of Manager employees for the Afternoon shift.
- $x_{27}$: Number of Security employees for the Afternoon shift.
- $x_{31}$: Number of Regular employees for the Night shift.
- $x_{32}$: Number of Part-Time employees on vacation during the Night shift.
- $x_{33}$: Number of Temporary employees on vacation during the Night shift.
- $x_{34}$: Number of On-call employees on vacation during the Night shift.
- $x_{35}$: Number of Training employees on training during the Night shift.
- $x_{36}$: Number of Manager employees on training during the Night shift.
- $x_{37}$: Number of Security employees on training during the Night shift.


### Constraints:


1. Demand constraint:

2. Requisites constraint:

3. Training regular constraint:

4. Managers constraint:

5. Security constraint:



$$X_{7s}, X_{8s}, X_{9s} \leq U_{\text{Part-time}}$$

$$X_{10s}, X_{11s}, X_{12s} \leq U_{\text{Temporary}}$$


4. Non-negativity constraints:

$$ X_{ij} \geq 0 \quad(\text{for all}\; i\in I,\; j\in J)$$ 


In [1]:
import pandas as pd
import numpy as np
from pyomo.environ import * 

In [2]:
model = ConcreteModel()

model.i = Set(initialize = ['Morning', 'Afternoon', 'Night'], doc = 'Shift')

# The types of different Employee
model.j = Set(initialize = ['Regular', 'PartTime', 'Temp', 'OnCall', 'Training', 'Managers', 'Security'], doc = 'EmployeeType')




# Total amount of workers needed on a shift -> DEMAND
model.total_workers_needed_onshift = Param(model.i, initialize= {'Morning':30, 'Afternoon': 30, 'Night': 30}) 

# Total amount of different workers we have for each type (each are independent - a worker of one type cannot be of another type) -> AVAILABLITY
model.indiv_workers_have = Param(model.j, initialize={'Regular': 50, 'PartTime': 10, 'Temp': 10, 
                                                      'OnCall': 40, 'Training': 50, 'Managers': 10, 'Security': 5})


# Cost of each worker for each shift.
costperworkershift = {

    # total 21 decision variables
    ('Morning', 'Regular'): 30,
    ('Morning', 'PartTime'): 20,
    ('Morning', 'Temp'): 15,
    ('Morning', 'OnCall'): 25,
    ('Morning', 'Training'): 10,
    ('Morning', 'Managers'): 55,
    ('Morning', 'Security'): 40,

    ('Afternoon', 'Regular'): 30,
    ('Afternoon', 'PartTime'): 20,
    ('Afternoon', 'Temp'): 15,
    ('Afternoon', 'OnCall'): 25,
    ('Afternoon', 'Training'): 10,
    ('Afternoon', 'Managers'): 55,
    ('Afternoon', 'Security'): 40,

    ('Night', 'Regular'): 50,
    ('Night', 'PartTime'): 20,
    ('Night', 'Temp'): 15,
    ('Night', 'OnCall'): 30,
    ('Night', 'Training'): 10,
    ('Night', 'Managers'): 60,
    ('Night', 'Security'): 60,
 

}

# Cost function using the above variable costperworkershift
model.Cost = Param(model.i, model.j, initialize = costperworkershift, doc = 'cost of queries')

In [3]:
# Decision variables we want to solve for
# must be NonNegative since we have that condition (trivial constraint) and must be integers
# (cannot have a decimal amount of a person)

model.x = Var(model.i, model.j, doc = 'Planning', within=NonNegativeIntegers)


# we want to minimize the total cost in a day
def objective_rule(model):
    return sum(model.Cost[i,j]*model.x[i,j] for i in model.i for j in model.j)

model.objective = Objective(rule = objective_rule, sense = minimize, doc = 'cost per day')

In [4]:
# Demand on the shift. Each shift has different demands
def demand(model, i):

    # i = shift, j = employee type 
    # we are adding all the number of employees to be the amount needed on that shift
    return sum(model.x[i,j] for j in model.j)>=model.total_workers_needed_onshift[i]

model.demand = Constraint(model.i, rule = demand, doc= 'employee demand during shift')

In [5]:
# make sure that we are within the constraint of the amount of employees we have
def requisites(model, j):

    # i = shift, j = employee type 
    # we are adding all the number of employees to be the amount needed on that shift
    return sum(model.x[i,j] for i in model.i)<=model.indiv_workers_have[j]

model.reqs = Constraint(model.j, rule = requisites, doc= 'employee of shift type we have')

In [6]:
# If we choose a training, we must also have a regular to accompany and train them (1 regular per training)

def training_regular(model, i):
    return model.x[i, 'Training'] <= model.x[i, 'Regular']

model.train_reg = Constraint(model.i, rule = training_regular, doc = 'Each training accompanied by a Regular')

In [7]:
# We need at least 3 managers at all shifts:
def managers_3(model, i):
    return model.x[i, 'Managers'] >= 3

model.managers_3 = Constraint(model.i, rule = managers_3, doc = 'Must have at least 3 managers')

In [8]:
# We need at least 1 Security per shift
def security(model, i):

    # i = shift, j = employee type
    return model.x[i, 'Security'] >= 1

model.security = Constraint(model.i, rule = security, doc = 'Must have at least 1 Security per shift')

In [9]:
# Define a solver 
Solver = SolverFactory('glpk')

# Obtain the solution
Results = Solver.solve(model)

# Display the solution
model.x.display()

x : Planning
    Size=21, Index=x_index
    Key                       : Lower : Value : Upper : Fixed : Stale : Domain
    ('Afternoon', 'Managers') :     0 :   3.0 :  None : False : False : NonNegativeIntegers
      ('Afternoon', 'OnCall') :     0 :   0.0 :  None : False : False : NonNegativeIntegers
    ('Afternoon', 'PartTime') :     0 :   0.0 :  None : False : False : NonNegativeIntegers
     ('Afternoon', 'Regular') :     0 :  13.0 :  None : False : False : NonNegativeIntegers
    ('Afternoon', 'Security') :     0 :   1.0 :  None : False : False : NonNegativeIntegers
        ('Afternoon', 'Temp') :     0 :   0.0 :  None : False : False : NonNegativeIntegers
    ('Afternoon', 'Training') :     0 :  13.0 :  None : False : False : NonNegativeIntegers
      ('Morning', 'Managers') :     0 :   3.0 :  None : False : False : NonNegativeIntegers
        ('Morning', 'OnCall') :     0 :   0.0 :  None : False : False : NonNegativeIntegers
      ('Morning', 'PartTime') :     0 :   0.0 :  None

In [10]:
# Total amount of money spent
model.objective()


2220.0

Now we can do this more systematically. Placing all the previous code on the creation of the model into a function. This way we can simulate an entire week's schedule depending on the demand per shift each day. To do so it is also convenient to create a dataframe for the solution to the model including in the rows, the shift type and the column the employee type. Where the actual data of the model is the number of employees chosen for the shift type and employee type (decision variables solution).

It is done in the following way:

1. def obtain_df_solution(model) - takes in the model and converts the results into a dataframe.

2. def ComputeModel(demandonday:list) - takes in a list for the demand on each shift type (Morning, Afternoon, Night).

In [25]:
def obtain_df_solution(model):
    
    names = ['Morning', 'Afternoon', 'Night']
    shifts = ['Regular', 'PartTime', 'Temp', 'OnCall', 'Training', 'Managers', 'Security']
    # values = []

    index_order = list(model.x.index_set())
    # print(index_order)
    values_in_order = [model.x[i]() for i in index_order]

    values_array = np.empty((0, 7))

    # appending all the values of how many employee types to get for a shift
    # for v in model.component_data_objects(Var, active=True):
    #     values.append(v.value)

    # morning_nparray = values[0:len(shifts)]
    # afternoon_nparray = values[len(shifts):len(shifts)*2]
    # night_nparray = values[len(shifts)*2, len(shifts)*3]

    for i in range(0,len(names)):
        values_array = np.append(values_array, [values_in_order[len(shifts)*i:len(shifts)*(i+1)]], axis=0)

 

    # creating the df
    df = pd.DataFrame(values_array, index=names, columns=shifts)


    Solver = SolverFactory('glpk')

    # Obtain the solution

    return df

Insert the model into a Pandas DataFrame

In [26]:
df = obtain_df_solution(model)
df

,Regular,PartTime,Temp,OnCall,Training,Managers,Security
Morning,13.0,0.0,0.0,0.0,13.0,3.0,1.0
Afternoon,13.0,0.0,0.0,0.0,13.0,3.0,1.0
Night,0.0,10.0,10.0,6.0,0.0,3.0,1.0


In [14]:
def ComputeModel(demandonday:list):

    # demandonday represents a list containing the demands on all shifts per day
    # if we are to use the above list then all we need to change is the
    # model.total_workers_needed_onshift and initialize it with the 
    # specific numbers per shift.

    # Therefore, we need to create a dictionary for this


    # example demandonday = [30, 30, 30]


    # Then create a list of names for the shifts
    names = ['Morning', 'Afternoon', 'Night']

    # create the dictionary needed with a comprehension format
    initializeworkersneeded = {str(key):value for key,value in zip(names,demandonday)}


    model = ConcreteModel()

    # Insert your code below
    #  'Tuesday', 'Wednesday', 'Thursday', 'Friday'
    model.i = Set(initialize = names, doc = 'Shift')

    # The types of different Employee
    model.j = Set(initialize = ['Regular', 'PartTime', 'Temp', 'OnCall', 'Training', 'Managers', 'Security'], doc = 'EmployeeType')

    # total amount of workers needed on a shift
    model.total_workers_needed_onshift = Param(model.i, initialize= initializeworkersneeded) # we need 'a' workers total per shift

    # Total amount of different workers we have for each type (each are independent - a worker of one type cannot be of another type).
    model.indiv_workers_have = Param(model.j, initialize={'Regular': 50, 'PartTime': 10, 'Temp': 10, 
                                                        'OnCall': 40, 'Training': 50, 'Managers': 10, 'Security': 5})


    # cost of each worker for each shift.
    costperworkershift = {

        # total 21 decision variables
        ('Morning', 'Regular'): 30,
        ('Morning', 'PartTime'): 20,
        ('Morning', 'Temp'): 15,
        ('Morning', 'OnCall'): 25,
        ('Morning', 'Training'): 10,
        ('Morning', 'Managers'): 55,
        ('Morning', 'Security'): 40,


        ('Afternoon', 'Regular'): 30,
        ('Afternoon', 'PartTime'): 20,
        ('Afternoon', 'Temp'): 15,
        ('Afternoon', 'OnCall'): 25,
        ('Afternoon', 'Training'): 10,
        ('Afternoon', 'Managers'): 55,
        ('Afternoon', 'Security'): 40,


        ('Night', 'Regular'): 50,
        ('Night', 'PartTime'): 20,
        ('Night', 'Temp'): 15,
        ('Night', 'OnCall'): 30,
        ('Night', 'Training'): 10,
        ('Night', 'Managers'): 60,
        ('Night', 'Security'): 60,

    }

    # Cost function using the above costperworkershift
    model.Cost = Param(model.i, model.j, initialize = costperworkershift, doc = 'cost of queries')

    # Decision variables we want to solve for
    # must be NonNegative since we have that condition (trivial constraint) and must be integers
    # (cannot have a decimal amount of a person)

    model.x = Var(model.i, model.j, doc = 'Planning', within=NonNegativeIntegers)


    # we want to minimize the total cost in a day
    def objective_rule(model):
        return sum(model.Cost[i,j]*model.x[i,j] for i in model.i for j in model.j)

    model.objective = Objective(rule = objective_rule, sense = minimize, doc = 'cost per day')

    def demand(model, i):

        # i = shift, j = employee type 
        # we are adding all the number of employees to be the amount needed on that shift
        return sum(model.x[i,j] for j in model.j)>=model.total_workers_needed_onshift[i]

    model.demand = Constraint(model.i, rule = demand, doc= 'employee demand during shift')

    def requisites(model, j):

        # i = shift, j = employee type 
        # we are adding all the number of employees to be the amount needed on that shift
        return sum(model.x[i,j] for i in model.i)<=model.indiv_workers_have[j]

    model.reqs = Constraint(model.j, rule = requisites, doc= 'employee of shift type we have')

    # If we choose a training, we must also have a regular to accompany and train them (1 regular per training)

    def training_regular(model, i):

        return model.x[i, 'Training'] <= model.x[i, 'Regular']

    model.train_reg = Constraint(model.i, rule = training_regular, doc = 'Each training accompanied by a Regular')

    # We need at least 3 managers at all shifts:
    def managers_3(model, i):
        return model.x[i, 'Managers'] >= 3

    model.managers_3 = Constraint(model.i, rule = managers_3, doc = 'Must have at least 3 managers')

    # We need at least 1 Security per shift
    def security(model, i):

        # i = shift, j = employee type
        return model.x[i, 'Security'] >= 1

    model.security = Constraint(model.i, rule = security, doc = 'Must have at least 1 Security per shift')

    Results = Solver.solve(model)
    
    obj = model.objective()

    
    return model,obj

In [ ]:
def addcost(df:pd.DataFrame):
    # gets a pandas dataframe and adds the cost per shift (new column)

    return

In [34]:
demandMonday  = [30,30,30]
modelMonday,objectiveMonday = ComputeModel(demandMonday)
dfMonday = obtain_df_solution(modelMonday)


demandTuesday= [60,50,50]
modelTuesday, objectiveTuesday  = ComputeModel(demandTuesday)
dfTuesday= obtain_df_solution(modelTuesday)


demandWednesday= [60,50,50]
modelWedn, objectiveWedn  = ComputeModel(demandWednesday)
dfWedn= obtain_df_solution(modelWedn)

demandThur= [60,50,50]
modelThur, objectiveThur  = ComputeModel(demandThur)
dfThur= obtain_df_solution(modelThur)

demandFri= [60,50,50]
modelFri, objectiveFri  = ComputeModel(demandFri)
dfFri= obtain_df_solution(modelFri)

demandSat= [60,50,50]
modelSat, objectiveSat  = ComputeModel(demandSat)
dfSat= obtain_df_solution(modelSat)

demandSun= [60,50,50]
modelSun, objectiveSun  = ComputeModel(demandSun)
dfSun= obtain_df_solution(modelSun)



# 




---
---

# Todo


- Even out the cost of each employee per shift


- Plot for each day of the week a calender for the employees


## Part C: Sensitivities associated with each constraint, and interpretion of the obtained values.

### Part D: Binary Variables:

Modify the problem in (a) to impose some logical and conditional constraints that require the use of binary or integer variables. The more integer variables or constraints, the better. Implement and solve this new model and interpret the results.

 Employee | Monday | Tuesday | Wednesday | Thursday | Friday | Saturday | Sunday | Profit
 -------|--------|--|--|---------
  1 |  9 | 9 | 9  | |6  |7   8|| 
  2 |  4 | 7 | 10 | 40 |
  3 |  2 | 3 | 4  | 5 | 6 | 7 | 8|   4 |  7 | 4 | 1  |  15
  4 |  2 | 3 | 4  | 5 | 6 | 7 | 8| 
  5 |  2 | 3 | 4  | 5 | 6 | 7 | 8| 
  6 |  2 | 3 | 4  | 5 | 6 | 7 | 8| 
1 |  2 | 3 | 4  | 5 | 6 | 7 | 8| 



| $_\text{Employee Type}$  $^{\text{Day of week}}$ | Monday | Tuesday | Wednesday | Thursday | Friday | Saturday | Sunday | Profit |
|---------------|---------|-----------|-------|-----|-----|-----|-----|-----|
| 1- Regular       | 15 | 15 | 15 |15 | 20 | 20 | 20 | 200 |  
| 2- Part-time     | 10 | 10 | 10 | 10 | 15 | 15 | 15 | 130|  
| 3- Temporary     | 8 | 8 | 8 | 8 | 12 | 12 | 12 | 115 | 
| 4- On-call       | 9 | 9 | 9 | 9 | 13 | 13 | 13 | 95 |  
| 5- Training      | 4 | 4 | 4 |4 |8 | 8 | 8 | 80 |  
| 6- Manager       | 18 | 18 | 18 | 18 | 18 | 23 | 23 | 225|    


In [160]:



from pyomo.environ import *
import gurobi
model = ConcreteModel()

# variables
model.x = Var([1,2,3,4,5,6], domain=Binary) # NOTE the domain = Binary

# objective function --> DONT FORGET THE sense=maximize
model.OBJ = Objective(expr=200*model.x[1] + 130*model.x[2] + 115*model.x[3] + 95*model.x[4] + 80*model.x[5] + 225*model.x[6], sense=maximize)

# constraints

# constraint for each days of the week 1-7(Mon-Sun)

# Monday
model.cons1 = Constraint(expr=15*model.x[1] + 10*model.x[2] + 8*model.x[3] + 9*model.x[4] + 4*model.x[5] + 20*model.x[6] <=50)
# Tuesday
model.cons2= Constraint(expr=15*model.x[1] + 10*model.x[2] + 8*model.x[3] + 9*model.x[4] + 4*model.x[5]+ 20*model.x[6] <=50)
# Wednesday
model.cons3 = Constraint(expr=15*model.x[1] + 10*model.x[2] + 8*model.x[3] + 9*model.x[4] + 4*model.x[5]+ 20*model.x[6] <=50)
# Thursday
model.cons4 = Constraint(expr=15*model.x[1] + 10*model.x[2] + 8*model.x[3] + 9*model.x[4] + 4*model.x[5]+ 20*model.x[6] <=50)
# Friday
model.cons5 = Constraint(expr=20*model.x[1] + 15*model.x[2] + 12*model.x[3] + 13*model.x[4] + 8*model.x[5]+ 25*model.x[6] <=80)
# Saturday
model.cons6 = Constraint(expr=20*model.x[1] + 15*model.x[2] + 12*model.x[3] + 13*model.x[4] + 8*model.x[5]+ 25*model.x[6] <=80)
# Sunday
model.cons7 = Constraint(expr=20*model.x[1] + 15*model.x[2] + 12*model.x[3] + 13*model.x[4] + 8*model.x[5]+ 25*model.x[6] <=80)




Solver = SolverFactory('glpk')

# Solver = SolverFactory('gurobi')

Results = Solver.solve(model)

# Display solution
display(model.OBJ)


OBJ : Size=1, Index=None, Active=True
    Key  : Active : Value
    None :   True : 635.0


In [144]:
display(model)

Model unknown

  Variables:
    x : Size=6, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          1 :     0 :   1.0 :     1 : False : False : Binary
          2 :     0 :   1.0 :     1 : False : False : Binary
          3 :     0 :   0.0 :     1 : False : False : Binary
          4 :     0 :   0.0 :     1 : False : False : Binary
          5 :     0 :   1.0 :     1 : False : False : Binary
          6 :     0 :   1.0 :     1 : False : False : Binary

  Objectives:
    OBJ : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True : 635.0

  Constraints:
    cons1 : Size=1
        Key  : Lower : Body : Upper
        None :  None : 49.0 :  50.0
    cons2 : Size=1
        Key  : Lower : Body : Upper
        None :  None : 49.0 :  50.0
    cons3 : Size=1
        Key  : Lower : Body : Upper
        None :  None : 49.0 :  50.0
    cons4 : Size=1
        Key  : Lower : Body : Upper
        None :  None : 49.0 :  50.0
    cons5 : Size=1

# Bibliography